# Search within 10 verses

Simple notebook POC for searching with in 10 Bible verses.

In [120]:
vers = [
    '我靠主大大地喜乐，因为你们思念我的心如今又发生；你们向来就思念我，只是没得机会。',
    '应当一无挂虑，只要凡事借着祷告、祈求，和感谢，将你们所要的告诉　神。',
    '我在急难中求告耶和华，向我的　神呼求。他从殿中听了我的声音；我在他面前的呼求入了他的耳中。',
    '　神啊，我曾求告你，因为你必应允我；求你向我侧耳，听我的言语。',
    '人的愚昧倾败他的道；他的心也抱怨耶和华。',
    '耶稣对他说：「你若能信，在信的人，凡事都能。」',
    '约书亚对以色列人说：「耶和华－你们列祖的　神所赐给你们的地，你们耽延不去得，要到几时呢？',
    '所罗门说：「耶和华－以色列的　神是应当称颂的！因他亲口向我父大卫所应许的，也亲手成就了。',
    '「因为你富有的时候，不欢心乐意地事奉耶和华－你的　神，',
    '你们当乐意事奉耶和华，当来向他歌唱！'
]
searches = ['挂虑 祈祷', '喜乐 事奉', '求救', '信心 行事']

## Tokenize w/o Stopwords

In [2]:
import jieba
import jieba.posseg as pseg

jieba.enable_paddle()

Paddle enabled successfully......


In [103]:
tokens = list(map(lambda v: pseg.lcut(v, use_paddle=True), vers))
for tk in tokens:
    print(tk)

[pair('我', 'r'), pair('靠', 'p'), pair('主大大地喜乐，', 'nt'), pair('因为', 'p'), pair('你们', 'r'), pair('思念', 'v'), pair('我', 'r'), pair('的', 'u'), pair('心', 'n'), pair('如今', 't'), pair('又', 'd'), pair('发生', 'v'), pair('；', 'v'), pair('你们', 'r'), pair('向来', 'd'), pair('就', 'd'), pair('思念', 'v'), pair('我', 'r'), pair('，', 'v'), pair('只是', 'd'), pair('没得', 'v'), pair('机会', 'n'), pair('。', 'v')]
[pair('应当', 'v'), pair('一无', 'v'), pair('挂虑', 'vn'), pair('，', 'n'), pair('只要', 'c'), pair('凡事', 'n'), pair('借', 'v'), pair('着', 'u'), pair('祷告', 'v'), pair('、', 'n'), pair('祈求', 'v'), pair('，', 'n'), pair('和', 'c'), pair('感谢', 'v'), pair('，', 'n'), pair('将', 'p'), pair('你们', 'r'), pair('所', 'u'), pair('要', 'v'), pair('的', 'u'), pair('告诉', 'v'), pair('\u3000神。', 'nr')]
[pair('我', 'r'), pair('在', 'p'), pair('急难', 'a'), pair('中', 'f'), pair('求告', 'v'), pair('耶和华，', 'PER'), pair('向', 'p'), pair('我', 'r'), pair('的', 'u'), pair('\u3000神', 'n'), pair('呼', 'v'), pair('求', 'v'), pair('。', 'v'), pair('他', 'r'), pai

## Stem / Lemmatize

In [4]:
import nltk
from nltk.corpus import wordnet as wn

nltk.download("wordnet")
nltk.download('omw')

[nltk_data] Error loading wordnet: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>
[nltk_data] Error loading omw: <urlopen error [Errno 8] nodename nor
[nltk_data]     servname provided, or not known>


False

### Lemmatize

In [5]:
syns = wn.synsets('感谢', pos='v', lang='cmn')
for syn in syns:
    print(syn.lemma_names(lang='cmn'))

['感谢', '谢意', '谢谢', '道谢']
['感激', '感谢']


### Similarity

In [167]:
import itertools

def similarity(token_a, token_b) -> float:
    word_a, pos_a = token_a
    if type(token_b) is str:
        word_b = token_b
        pos_b = None
    else:
        word_b, pos_b = token_b
    if word_a == word_b:
        return 1

    if pos_a:
        pos_a = map_pos(pos_a)
    if pos_b:
        pos_b = map_pos(pos_b)

    asyns = wn.synsets(word_a, pos=pos_a, lang='cmn')
    bsyns = wn.synsets(word_b, pos=pos_b, lang='cmn')
    if asyns and bsyns:
        sims = map(lambda p: wn.path_similarity(*p), itertools.product(asyns, bsyns))
    elif asyns:
        lemmas = (syn.lemma_names(lang='cmn') for syn in asyns)
        sims = [(1 if word_b == lemma else 0) for lemma in lemmas]
    elif bsyns:
        lemmas = (syn.lemma_names(lang='cmn') for syn in bsyns)
        sims = [(1 if word_a == lemma else 0) for lemma in lemmas]
    else:
        sims = [0]
    return max(sims)

def map_pos(pos: str) -> str:
    wn_pos = {
        'n': wn.NOUN,
        'v': wn.VERB,
        'a': wn.ADJ,
        'd': wn.ADV
    }
    return wn_pos.get(pos[0]) or pos

In [172]:
tests = [
    similarity(('感谢', 'v'), '感动'),
    similarity(('感谢', 'v'), '谢意'),
    similarity(('感谢', 'v'), '谢谢'),
    similarity(('挂虑', 'v'), '挂念'),
    similarity(('挂虑', 'v'), '挂虑'),
    similarity(('祷告', 'v'), '祈求')
]
print(tests)

[0.1, 1.0, 1.0, 0, 1, 0.07142857142857142]


In [169]:
def remove_stopwords(tokens):
    return [(word, pos) for word, pos in tokens if pos != 'x' and word not in '。，；⋯⋯、']

print(tokens[0])
print(remove_stopwords(tokens[0]))

[pair('我', 'r'), pair('靠', 'p'), pair('主大大地喜乐，', 'nt'), pair('因为', 'p'), pair('你们', 'r'), pair('思念', 'v'), pair('我', 'r'), pair('的', 'u'), pair('心', 'n'), pair('如今', 't'), pair('又', 'd'), pair('发生', 'v'), pair('；', 'v'), pair('你们', 'r'), pair('向来', 'd'), pair('就', 'd'), pair('思念', 'v'), pair('我', 'r'), pair('，', 'v'), pair('只是', 'd'), pair('没得', 'v'), pair('机会', 'n'), pair('。', 'v')]
[('我', 'r'), ('靠', 'p'), ('主大大地喜乐，', 'nt'), ('因为', 'p'), ('你们', 'r'), ('思念', 'v'), ('我', 'r'), ('的', 'u'), ('心', 'n'), ('如今', 't'), ('又', 'd'), ('发生', 'v'), ('你们', 'r'), ('向来', 'd'), ('就', 'd'), ('思念', 'v'), ('我', 'r'), ('只是', 'd'), ('没得', 'v'), ('机会', 'n')]


In [170]:
def sentence_similarity(keywords, sentence):
    keywords = remove_stopwords(keywords)
    sentence = remove_stopwords(sentence)
    keywords_similarity = (max(similarity(kw, token) for token in sentence) for kw in keywords)
    return sum(keywords_similarity) / len(keywords)

search_tk = pseg.lcut(searches[0])
print(sentence_similarity(search_tk, tokens[0]))
print(sentence_similarity(search_tk, tokens[1]))

0.07142857142857142
1.0


## Jaccard Similarity

In [171]:
# Reference: https://bommaritollc.com/2014/06/12/fuzzy-match-sentences-python/
# https://github.com/fxsjy/jieba